# Classification:

While Linear Regression is used to predict a numeric value, classification is used to separate data into different labels. 

In this example, I will use the TensorFlow estimator to classify flowers. 

Sourced mainly from: https://www.tensorflow.org/tutorials/estimator/premade

Note that there won't be as many notes, as most of it is covered in previous lessons (and are therefore in the previous Jupyter Notebook)

# Imports and Setup:

In [22]:
#importing various libraries into this session:

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd



# Data Set:
This specific dataset will seperate flowers into 3 different classes/species:
* Setosa
* Versicolor
* Virginica

Information is given for each flower, this includes:
* Sepal Length
* Sepal Width
* Petal Length
* Petal Width

In [23]:
#Creating some constants:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [24]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)


#Here we are using Keras (a module from within TensorFlow) to grab the datasets and read them into a pandas dataframe.

Time to have a look at the data:

In [25]:
train.head() #First 5 lines of the dataframe

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


For each of the datasets, split out the labels, which the model will be trained to predict.

In [26]:
train_y = train.pop('Species')
test_y = test.pop('Species')

# The label column has now been removed from the features.
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


Looking at the shape to make sure it has the expected amount of entries:

In [27]:
train.shape # Expected to have 120 entries (i.e. rows) + 4 features (i.e. columns)

(120, 4)

# Input Function:

This input function is a little easier to understand compared to the one used in the Linear Regression example.

In [28]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

# Feature Columns:

Here I'll put in the feature columns.

In [29]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


# Building the model:
Now you are able to choose a model. For Classification, there are a few estimators/models that you can pick from, some examples are:
* ``tf.estimator.DNNClassifier`` for deep models that perform multi-class classification.
* ``tf.estimator.DNNLinearCombinedClassifier`` for wide & deep models.
* ``tf.estimator.LinearClassifier`` for classifiers based on linear models.

You are able to choose either from a deep neural network (DDN) or from a Linear model. As there is unlikely to be a linear relationship, it makes sense to use the deep neural network for this use-case.

In [32]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpxr8hkkkt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
